In [1]:
import numpy as np
from scipy.stats import ortho_group
from embedding.schema import Schema
from embedding.encoder import Encoder
from embedding.structure import Struct

In [117]:
schema = Schema(labels=['A1','A2','a','b','c','d'], attributes=['A1', 'A2'])

n_emb = 1024
encoder = Encoder(schema, dim=n_emb)
E = encoder.token_emb

In [118]:
def attention(seq, temp=100):
    """Causal attention over a sequence of vectors."""
    q = [Z.T@s[0] for s in seq]
    k = [s[0] for s in seq]
    v = [s[2] for s in seq]
    for i, s in enumerate(seq):
        res = np.zeros((i,))
        for j, ss in enumerate(seq[:i]):
            res[j] = q[i].T @ k[j]
        res *= temp
        w = np.exp(res)/np.sum(np.exp(res))
        if i != 0:
            seq[i][1] = np.sum([w[j]*v[j] for j in range(i)], axis=0)


def relu(x):
    return np.maximum(x, 0)


def ff_decode(x, C=100):
    """Feed forward decoder layer"""
    I = np.ones(E.shape[0])
    cond = C*(E @ x - .5 * I)
    return E.T@(relu(cond + I) - relu(cond))


def ff_path_decode(x, p, C=100):
    """Feed forward path decoder layer"""
    n_attr = encoder.attr_emb.shape[0]
    I = np.ones(n_attr)
    E_attr = E[:n_attr]
    cond = C*(E_attr @ p - .5 * I).repeat(n_emb).reshape(n_attr, n_emb)
    trans = np.array([encoder.attr_emb[i].T@x for i in range(n_attr)])

    return x + np.sum(relu(cond + trans - x) - relu(cond), axis=0)


def ff(seq):
    """Apply feed forward decoder to a sequence of vectors."""
    for s in seq:
        s[2] = ff_path_decode(s[1], s[3])
        s[4] = ff_decode(s[2])

In [119]:
## Initialize seq with positional embedding
p_dim = 16
Z = ortho_group.rvs(p_dim)
p = np.random.normal(size=p_dim)
p /= np.linalg.norm(p)

n_seq = 16
seq = []
for i in range(n_seq):
    seq.append([p,  # position 
                np.zeros(n_emb), # vector
                np.zeros(n_emb), # transformed vector
                np.random.normal(size=n_emb), # path
                np.zeros(n_emb)  # token
                ])
    p = Z @ p



## Place encoded vector in first position
x = Struct.create(schema, ('c', {'A1': ('b', {'A2': ('a', {'A1': ('d', {'A2':'a'})})})}))
v = encoder.encode(x)
seq[0][1] = v
seq[0][3] = np.zeros_like(v)
seq[1][3] = E[0]
seq[2][3] = E[1]
seq[3][3] = E[0]
seq[4][3] = E[1]


In [121]:
## Run transformer
for _ in range(n_seq):
    ff(seq)
    attention(seq)


[encoder.decode(s[4]) for s in seq]

["4",
 "3",
 "2",
 "5",
 "2",
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]